In [ ]:
import pandas as pd
import openai
import time


In [2]:
data = pd.read_csv('Preprocessed Data.csv')
data.head()

,Text,Link,Câu trả lời,TTHC,Ministry,Câu_hỏi_sạch,Câu_trả_lời_sạch,Tên_TTHC,Tên_cơ_quan_thực_hiện
0,Cá nhân đăng ký Bồi dưỡng nghiệp vụ đăng kiểm ...,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,"Theo quy định tại Khoản 3, Điều 8 Thông tư số ...","{'Tên': ['Cấp, cấp lại thẻ, dấu kỹ thuật đăng ...",Bộ Nông nghiệp và Phát triển nông thôn,cá nhân đăng ký bồi dưỡng nghiệp vụ đăng kiểm ...,theo quy định tại khoản 3 điều 8 thông tư số 2...,"['Cấp, cấp lại thẻ, dấu kỹ thuật đăng kiểm viê...",[['Cục Thủy sản - Bộ NN-PTNT']]
1,"Khi nào các tổ chức, cá nhân có thể gửi đề xuấ...",https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,Định kỳ hàng năm theo thông báo trên trang thô...,{'Tên': ['Phê duyệt danh mục đề tài khoa học v...,Bộ Nông nghiệp và Phát triển nông thôn,khi nào các tổ chức cá nhân có thể gửi đề xuất...,định kỳ hàng năm theo thông báo trên trang thô...,['Phê duyệt danh mục đề tài khoa học và công n...,[['Vụ khoa học công nghệ và môi trường']]
2,Điều kiện công nhận đối với TBKT công nghệ sin...,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,Theo Điều 6 Thông tư số 23/2010/TT-BNNPTNT ngà...,{'Tên': ['Công nhận tiến bộ kỹ thuật công nghệ...,Bộ Nông nghiệp và Phát triển nông thôn,điều kiện công nhận đối với tbkt công nghệ sin...,theo điều 6 thông tư số 23/2010/tt bnnptnt ngà...,['Công nhận tiến bộ kỹ thuật công nghệ sinh học'],[['Vụ khoa học công nghệ và môi trường']]
3,Cách thức thực hiện việc gửi đề xuất nhiệm vụ ...,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,Đề xuất nhiệm vụ KHCN có thể gửi trực tiếp tại...,{'Tên': ['Phê duyệt danh mục đề tài khoa học v...,Bộ Nông nghiệp và Phát triển nông thôn,cách thức thực hiện việc gửi đề xuất nhiệm vụ ...,đề xuất nhiệm vụ khcn có thể gửi trực tiếp tại...,['Phê duyệt danh mục đề tài khoa học và công n...,[['Vụ khoa học công nghệ và môi trường']]
4,Hồ sơ đề nghị thu hồi Giấy chứng nhận cơ sở đủ...,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,"Theo quy định tại Khoản 4, Điều 54 của Nghị đị...","{'Tên': ['Cấp, cấp lại giấy chứng nhận cơ sở đ...",Bộ Nông nghiệp và Phát triển nông thôn,hồ sơ đề nghị thu hồi giấy chứng nhận cơ sở đủ...,theo quy định tại khoản 4 điều 54 của nghị địn...,"['Cấp, cấp lại giấy chứng nhận cơ sở đủ điều k...",[['Cơ quan quản lý nhà nước về thủy sản cấp tỉ...


In [ ]:
client = openai.OpenAI(api_key="your-api-key-here")

In [ ]:
instruction = "Tôi muốn bạn đóng vai trò như một trình tạo câu trả lời sai (hallucinated answer generator). Đưa ra một câu hỏi, câu trả lời đúng, và chủ đề liên quan, mục tiêu của bạn là viết một câu trả lời nghe có vẻ hợp lý nhưng thực tế lại sai. Bạn đang cố gắng trả lời câu hỏi nhưng có một sự mâu thuẫn thực tế giữa câu trả lời và kiến thức. Bạn có thể bịa ra một số thông tin không có trong kiến thức được cung cấp. Bạn chỉ trả lời mà không nhắc lại hay lưu ý bất kỳ điều gì."

In [ ]:
def get_qa_res(topic, question, answer, instruction):
    if isinstance(instruction, str):
        message = [
            {"role": "system", "content": instruction},  
            {"role": 
             "user", "content":
                                "\n\n#Chủ đề#: " + topic +
                                "\n#Câu hỏi#: " + question +
                                "\n#Câu trả lời đúng#: " + answer +
                                "\n#Câu trả lời bịa đặt#: "}
        ]
    else:
        raise TypeError("The instruction must be str or list!")
             
    while True:
        try:
            res = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=message,
                temperature=1,
                max_tokens=4000,
                top_p=1
            )
            break
        except openai.error.RateLimitError:
            print('openai.error.RateLimitError\nRetrying...')
            time.sleep(60)
        except openai.error.ServiceUnavailableError:
            print('openai.error.ServiceUnavailableError\nRetrying...')
            time.sleep(20)
        except openai.error.Timeout:
            print('openai.error.Timeout\nRetrying...')
            time.sleep(20)
        except openai.error.APIError:
            print('openai.error.APIError\nRetrying...')
            time.sleep(20)
        except openai.error.APIConnectionError:
            print('openai.error.APIConnectionError\nRetrying...')
            time.sleep(20)
    
    
    # print(res['choices'][0]['message']['content'])
    return res.choices[0].message.content

In [ ]:
new_data = {"Chủ đề": [], "Câu hỏi": [], "Câu trả lời đúng": [], "Câu trả lời bịa đặt": []}
def generate_qa_dataset(data, instruction):
    for i in range(10):
        question = data.iloc[i]['Text']
        answer = data.iloc[i]['Câu trả lời']
        topic = data.iloc[i]['Ministry']
        ans = get_qa_res(topic, question, answer, instruction)
        new_data["Chủ đề"].append(topic)
        new_data["Câu hỏi"].append(question)
        new_data["Câu trả lời đúng"].append(answer)
        new_data["Câu trả lời bịa đặt"].append(ans)
        print(" sample {} completed!".format(i))

In [ ]:
new_data_df =pd.DataFrame(new_data)
pd.to_csv('Generated Data.csv', new_data_df, enconde='utf-8')

In [ ]:
import json
json_data = json.dumps([dict(zip(new_data.keys(), values)) for values in zip(*new_data.values())], ensure_ascii=False, indent=4)
with open('Generated Data.json', 'w', encoding='utf-8') as f:
    f.write(json_data)